In [1]:
import pandas as pd
import os
from apiclient.discovery import build

api_key = '' #your key here

youtube = build('youtube', 'v3', developerKey=api_key)

y2k = pd.Timestamp(2000,1,1)

def fetch_date(vid_id):
    vid_snippet = youtube.videos().list(part='snippet', id=vid_id)
    vid_snippet = vid_snippet.execute()
    date =vid_snippet['items'][0]['snippet']['publishedAt']
    return pd.to_datetime(date).tz_localize(None)

In [2]:
directory = 'youtube_trends_stats_de/single_json/'
files = os.listdir(directory)
frames = []
for file in files:
    frames.append(pd.read_json(directory + file))
    if not 'dataTakenAt' in frames[-1]:
        frames[-1]['dataTakenAt'] = pd.to_datetime(file[:-5].replace('_',' '), utc='CEST').tz_localize(None)

In [3]:
for frame in frames:
    if(frame.dataTakenAt.dtype == 'int64'):
        frame.dataTakenAt = pd.to_datetime(frame.dataTakenAt, unit='ms')
    if(frame.publishedAt.dtype == 'int64'):
        frame.publishedAt = pd.to_datetime(frame.publishedAt, unit='ms')
data = pd.concat(frames)
data.publishedAt = data.publishedAt.map(lambda date : pd.to_datetime(date).tz_localize(None))
data.dataTakenAt = data.dataTakenAt.map(lambda date : pd.to_datetime(date).tz_localize(None))
ids = list(data.id.unique())
grouped_by_id = data.groupby('id')

In [4]:
videos = []
for id in ids:
    vid = grouped_by_id.get_group(id).sort_values(by='dataTakenAt')
    df = pd.DataFrame(vid.iloc[0]).transpose()
    df['maxRank'] = vid['rank'].min()
    df['minRank'] = vid['rank'].max()
    df['meanRank'] = vid['rank'].mean()
    
    if vid.publishedAt.iloc[0] < y2k:
        df.publishedAt = fetch_date(id)
        
    df['enteredTrends'] = vid.dataTakenAt.iloc[0]
    df['leftTrends'] = vid.dataTakenAt.iloc[-1]
    df['timeInTrends'] = df.leftTrends - df.enteredTrends
    df['timeUntilTrending'] = df['enteredTrends'] - df.publishedAt
    
    df['maxViews'] = vid['viewCount'].max()
    df['minViews'] = vid['viewCount'].min()
    df['meanViews'] = round(vid['viewCount'].mean())
    
    df['maxLikes'] = vid['likeCount'].max()
    df['minLikes'] = vid['likeCount'].min()
    df['meanLikes'] = round(vid['likeCount'].mean())
    
    df['maxDislikes'] = vid['dislikeCount'].max()
    df['minDislikes'] = vid['dislikeCount'].min()
    df['meanDislikes'] = round(vid['dislikeCount'].mean())
    
    df['maxComments'] = vid['commentCount'].max()
    df['minComments'] = vid['commentCount'].min()
    df['meanComments'] = round(vid['commentCount'].mean())
    videos.append(df)

videos = pd.concat(videos, ignore_index=True).sort_values(by='publishedAt')